In [70]:
import pandas as pd
import os
os.chdir('C:\\Users\\akhil\\Downloads')
mfi=pd.read_stata('KCC_cnt (1).dta')

In [71]:
mfi.columns
ru_type=mfi['ru_classification'].unique()
len_type=mfi['lendertype'].unique()
print(ru_type,len_type)

['' 'Semi-Urban' 'Rural' 'Urban'] ['PSU Banks' 'Private Banks' 'Others' 'NBFCs']


In [72]:
rural=[]
semi_urban=[]
urban=[]
others_len=[]
psu=[]
priv_banks=[]
nbfc=[]
for i in range(mfi.shape[0]):
    if(mfi.ru_classification.iat[i]=='Rural'):
        rural.append(1)
        semi_urban.append(0)
        urban.append(0)
    elif(mfi.ru_classification.iat[i]=='Urban'):
        rural.append(0)
        semi_urban.append(0)
        urban.append(1)
    elif(mfi.ru_classification.iat[i]=='Semi-Urban'):
        rural.append(0)
        semi_urban.append(1)
        urban.append(0)
    else:
        rural.append(0)
        semi_urban.append(0)
        urban.append(0)
len_type=['Others', 'PSU Banks', 'Private Banks', 'NBFCs']
for i in range(mfi.shape[0]):
    if(mfi.lendertype.iat[i]==len_type[0]):
        others_len.append(1)
        psu.append(0)
        nbfc.append(0)
        priv_banks.append(0)
    elif(mfi.lendertype.iat[i]==len_type[1]):
        others_len.append(0)
        psu.append(1)
        nbfc.append(0)
        priv_banks.append(0)
    elif(mfi.lendertype.iat[i]==len_type[2]):
        others_len.append(0)
        psu.append(0)
        nbfc.append(0)
        priv_banks.append(1)
    elif(mfi.lendertype.iat[i]==len_type[3]):
        others_len.append(0)
        psu.append(0)
        nbfc.append(1)
        priv_banks.append(0)
    else:
        others_len.append(0)
        psu.append(0)
        nbfc.append(0)
        priv_banks.append(0)


In [73]:
mfi['dummy_rural']=rural
mfi['urban']=urban
mfi['dummy_semi_urban']=semi_urban
mfi['dummy_psu']=psu
mfi['dummy_others_len']=others_len
mfi['dummy_priv']=priv_banks
mfi['dummy_nbfcs']=nbfc

In [74]:
border_markings=pd.read_csv('districts_correct.csv')

In [75]:
cols2=['night_lights', 'qrt_yr', 'district', 'rainfall', 'disbursal_period_actual', 'quarterofbookclosingperoid', 'state','ru_classification', 'lendertype', 'avgticketsizelakhs','val_def','volume_def','volume_default_90_plus', 'value_default_90_plus','int_ay', 'int_pmk', 'pre_lend','dummy_rural', 'urban', 'dummy_semi_urban', 'dummy_psu','dummy_others_len', 'dummy_priv', 'dummy_nbfcs']

In [76]:
mfi1=mfi[cols2]

In [77]:
mfi1=mfi1.merge(border_markings,on='district')

In [78]:
mfi_bord=mfi1[mfi1['border']==1]

In [79]:
df_new=mfi_bord[(mfi_bord['PB']==1)|(mfi_bord['HR']==1)]

In [80]:
df_new.state.unique()

array(['UP', 'RJ', 'HR', 'PB', 'HP', 'UK', 'JK', 'DL', 'CH'], dtype=object)

In [11]:
df_new.disbursal_period_actual.unique()

array(['FY 2017-2018 Q1', 'FY 2017-2018 Q3', 'FY 2017-2018 Q2',
       'BEFORE FY 2017-18', 'FY 2018-2019 Q2', 'FY 2018-2019 Q3',
       'OTHERS', 'FY 2017-2018 Q4', 'FY 2018-2019 Q1', 'FY 2018-2019 Q4',
       'FY 2019-2020 Q1', 'FY 2019-2020 Q2', 'FY 2019-2020 Q3',
       'FY 2019-2020 Q4'], dtype=object)

In [12]:
def get_dates(df):
    l=[]
    for i in range(df.shape[0]):
        
        year=df['qrt_yr'].iat[i][0:4]
        if(df['qrt_yr'].iat[i][4:]=='Q1'):
            l.append(year+'-01-01')
        elif(df['qrt_yr'].iat[i][4:]=='Q2'):
            l.append(year+'-04-01')
        elif(df['qrt_yr'].iat[i][4:]=='Q3'):
            l.append(year+'-07-01')
        elif(df['qrt_yr'].iat[i][4:]=='Q4'):
            l.append(year+'-10-01')
        else:
            l.append(None)
    df['date']=pd.to_datetime(l)    
    return df

In [83]:
df_new=get_dates(df_new)

<ipython-input-12-c5952187962d>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date']=pd.to_datetime(l)


In [84]:
post_new=[0]*df_new.shape[0]
pre=['2018Q2','2018Q3']
post=['2018Q4','2019Q1','2019Q2']
for i in range(df_new.shape[0]):
    if(df_new.qrt_yr.iat[i] in pre):
        post_new[i]=0
    if(df_new.qrt_yr.iat[i] in post):
        post_new[i]=1
    if(df_new.qrt_yr.iat[i] not in pre and df_new.qrt_yr.iat[i] not in post):
        post_new[i]=''
df_new['post_new']=post_new
df_new=df_new[df_new['post_new']!='']

<ipython-input-84-fa405cc1549d>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['post_new']=post_new


In [81]:
msp=[]
imp=['RJ', 'HP', 'UK', 'JK', 'DL', 'CH','UP']
for i in range(df_new.shape[0]):
    if(df_new.state.iat[i] in imp):
        msp.append(1)
    else:
        msp.append(0)
df_new['msp']=msp

<ipython-input-81-09d4a7dd255b>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['msp']=msp


In [85]:
df_new['int_msp']=df_new['msp']*df_new['post_new']

<ipython-input-85-2ab9014076e7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['int_msp']=df_new['msp']*df_new['post_new']


In [86]:
for i in range(df_new.shape[0]):
    if(df_new.state.iat[i]=='CH'):
        df_new.int_msp.iat[i]==1
    

In [90]:
df_new

,night_lights,qrt_yr,district,rainfall,disbursal_period_actual,quarterofbookclosingperoid,state,ru_classification,lendertype,avgticketsizelakhs,...,TN,TR,TS,UK,UP,WB,msp,date,post_new,int_msp
23442,1.330174,2018Q2,ALIGARH,0.029699,FY 2017-2018 Q2,Q2,UP,,PSU Banks,0.97,...,0,0,0,0,1,0,1,2018-04-01,0,0
23443,1.347421,2018Q3,ALIGARH,0.352317,BEFORE FY 2017-18,Q3,UP,Urban,PSU Banks,0.8,...,0,0,0,0,1,0,1,2018-07-01,0,0
23444,2.257817,2018Q4,ALIGARH,0.009392,FY 2017-2018 Q3,Q4,UP,Urban,PSU Banks,0.5,...,0,0,0,0,1,0,1,2018-10-01,1,1
23445,1.983886,2019Q1,ALIGARH,0.032161,FY 2018-2019 Q2,Q1,UP,Urban,Private Banks,0.31,...,0,0,0,0,1,0,1,2019-01-01,1,1
23446,1.439637,2019Q2,ALIGARH,0.015904,FY 2017-2018 Q2,Q2,UP,Semi-Urban,Private Banks,0.16,...,0,0,0,0,1,0,1,2019-04-01,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1335878,NaN,2019Q2,PATHANKOT,NaN,FY 2018-2019 Q1,Q2,PB,Urban,Private Banks,0.3,...,0,0,0,0,0,0,0,2019-04-01,1,0
1335879,NaN,2019Q2,PATHANKOT,NaN,FY 2017-2018 Q3,Q2,PB,Urban,Others,8.5,...,0,0,0,0,0,0,0,2019-04-01,1,0
1335880,NaN,2019Q2,PATHANKOT,NaN,FY 2017-2018 Q2,Q2,PB,Rural,Others,1.1,...,0,0,0,0,0,0,0,2019-04-01,1,0
1335881,NaN,2019Q2,PATHANKOT,NaN,FY 2019-2020 Q1,Q2,PB,Semi-Urban,PSU Banks,3.7,...,0,0,0,0,0,0,0,2019-04-01,1,0


In [19]:
def rem_common_control_dists(df):
    df1=df[(df['state']=='PB')&(df['HR']!=1)]
    df2=df[(df['state']=='HR')&(df['PB']!=1)]
    df3=df[(df['state']!='HR')&(df['state']!='PB')]
    df1=df1.append([df2,df3])
    return df1

In [27]:
def rest_disb(df,l,size):
    s='disbursal_period_actual'
    df=df[(df[s]==l[0])|(df[s]==l[1])|(df[s]==l[2])|(df[s]==l[3])|(df[s]==l[4])|(df[s]==l[5])|(df[s]==l[6])]
    
    if(size!=0):
        df=df[df['avgticketsizelakhs']!='']
        df['avgticketsizelakhs']=df['avgticketsizelakhs'].apply(float)
        df=df[df['avgticketsizelakhs']<size]
    
    return df

In [93]:
from linearmodels import PanelOLS

def vol_reg(df):
    df=df.set_index(['district','date'])
    df=df[df['ru_classification']!='']
    df=df[df['lendertype']!='']
    df['const']=1
    mod = PanelOLS(df['volume_default_90_plus'],df[['int_msp','dummy_rural', 'dummy_semi_urban', 'dummy_psu', 'dummy_priv','dummy_nbfcs','const']],entity_effects=True,time_effects=True)
    #mod = PanelOLS(df['volume_def'],df[['int_msp','rainfall','night_lights','dummy_rural', 'dummy_semi_urban', 'dummy_psu', 'dummy_priv','dummy_nbfcs','const']],entity_effects=True,time_effects=True)
    res = mod.fit(cluster_entity=True,cov_type='clustered')
    return res.summary
def val_reg(df):
    df=df.set_index(['district','date'])
    df=df[df['ru_classification']!='']
    df=df[df['lendertype']!='']
    df['const']=1
    mod = PanelOLS(df['value_default_90_plus'],df[['int_msp','rainfall','night_lights','dummy_rural', 'dummy_semi_urban', 'dummy_psu', 'dummy_priv','dummy_nbfcs','const']],entity_effects=True,time_effects=True)
    #mod = PanelOLS(df['val_def'],df[['int_msp','rainfall','night_lights','dummy_rural', 'dummy_semi_urban', 'dummy_psu', 'dummy_priv','dummy_nbfcs','const']],entity_effects=True,time_effects=True)
    res = mod.fit(cluster_entity=True,cov_type='clustered')
    return res.summary

In [104]:
l=['FY 2018-2019 Q1', 'BEFORE FY 2017-18', 'FY 2017-2018 Q3','FY 2017-2018 Q4', 'FY 2017-2018 Q2', 'FY 2017-2018 Q1','FY 2018-2019 Q2']
df1=rem_common_control_dists(df_new)
df2=rest_disb(df1,l,1.25)
vol_reg(df2)

C:\Users\akhil\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\linearmodels\shared\exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


Dep. Variable:,volume_default_90_plus,R-squared:,0.0605
Estimator:,PanelOLS,R-squared (Between):,-0.4404
No. Observations:,3420,R-squared (Within):,0.0600
Date:,"Thu, Apr 29 2021",R-squared (Overall):,0.0178
Time:,22:55:44,Log-likelihood,275.16
Cov. Estimator:,Clustered,,
,,F-statistic:,36.111
Entities:,44,P-value,0.0000
Avg Obs:,77.727,Distribution:,"F(6,3366)"
Min Obs:,9.0000,,
Max Obs:,337.00,F-statistic (robust):,1.3731


In [105]:
l=['FY 2018-2019 Q1', 'BEFORE FY 2017-18', 'FY 2017-2018 Q3','FY 2017-2018 Q4', 'FY 2017-2018 Q2', 'FY 2017-2018 Q1','FY 2018-2019 Q2']
df1=rem_common_control_dists(df_new)
df2=rest_disb(df1,l,1.25)
val_reg(df2)

C:\Users\akhil\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\linearmodels\shared\exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


Dep. Variable:,value_default_90_plus,R-squared:,0.0650
Estimator:,PanelOLS,R-squared (Between):,-0.5708
No. Observations:,3234,R-squared (Within):,0.0604
Date:,"Thu, Apr 29 2021",R-squared (Overall):,0.0027
Time:,22:55:54,Log-likelihood,225.18
Cov. Estimator:,Clustered,,
,,F-statistic:,27.636
Entities:,41,P-value,0.0000
Avg Obs:,78.878,Distribution:,"F(8,3181)"
Min Obs:,7.0000,,
Max Obs:,348.00,F-statistic (robust):,1.2589


In [92]:
df2.int_msp.value_counts()

0    22675
1    12827
Name: int_msp, dtype: int64